In [ ]:
import pydeck as pdk
import osmnx as ox
import pandas as pd

# Coordinates for the center of Bonn, Germany (latitude, longitude)
bonn_center = (50.7374, 7.0982)

#  Extract all buildings with height data
tags = {"building": True, "height": True}

buildings = ox.features_from_point(bonn_center, tags=tags, dist=500)


def get_valid_heights(buildings):
    """
    Extracts building polygons with valid height values from the OSMNx GeoDataFrame.

    Parameters:
        buildings (GeoDataFrame): OSM building footprints with attributes.

    Returns:
        DataFrame: A cleaned table containing:
                   - coordinates: list of polygon vertices
                   - height: numeric building height
    """
    data = []

    for idx, row in buildings.iterrows():
        height = row.get('height')

        
        if height and isinstance(height, (int, float, str)):
            try:
                height = float(height) 
                if height > 0:
                    # Get polygon coordinates from geometry field
                    coords = list(row.geometry.exterior.coords)
                    data.append({"coordinates": coords, "height": height})
            except:
                # Skip non-numeric heights
                continue

    return pd.DataFrame(data)


# Clean + filter building dataset
df = get_valid_heights(buildings)

# 3D polygon layer in PyDeck
layer = pdk.Layer(
    "PolygonLayer",
    df,
    get_polygon="coordinates",
    extruded=True,                    # Enable 3D height extrusion
    get_elevation="height",           # Use height column for extrusion
    get_fill_color=[180, 180, 180, 200],
    pickable=True,                    
)


view_state = pdk.ViewState(
    latitude=bonn_center[0],
    longitude=bonn_center[1],
    zoom=15,
    pitch=45
)

# Save the map to an HTML file
pdk.Deck(layers=[layer], initial_view_state=view_state).to_html(
    "Bonn_3d_real_heights.html"
)
